In [ ]:
import psutil
!pip install --upgrade imbalanced-learn scikit-learn

# Cell 1: Imports (add these lines)
!pip install shap
import shap
!pip install holidays
import holidays
!pip install openpyxl XlsxWriter

import gc
from typing import Dict, Any

import os
import numpy as np
import pandas as pd
import gc
import psutil
import os

import humanize
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif, VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import warnings
warnings.filterwarnings('ignore')

# Cell 2: Confirm if you're in high-RAM (should be >25GB)
import psutil
ram = psutil.virtual_memory()
print(f"✅ Available RAM: {ram.total / 1e9:.2f} GB")

import gc
import os
import joblib
import pyarrow.parquet as pq
from tqdm import tqdm
from datetime import datetime, timedelta
import optuna

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.combine import SMOTEENN, SMOTETomek

class AMEXClickPredictionPipeline:
    def __init__(self):
        self.scaler = StandardScaler()
        self.models = {}
        self.feature_cols = []
        self.feature_selector = None
        self.variance_selector = None
        self.target_encoders = {}
        self.temp_dir = "/kaggle/temp/"
        os.makedirs(self.temp_dir, exist_ok=True)

    def _clean_temp_files(self):
        """Remove intermediate files to save disk space"""
        for i in range(1, 8):
            file_path = f"{self.temp_dir}temp_step{i}.parquet"
            if os.path.exists(file_path):
                os.remove(file_path)

    def load_data(self):
        """Optimized data loading with memory optimization"""
        print("🔄 Loading datasets with memory optimization...")

        def get_optimized_dtypes(df):
            dtype_map = {}
            for col in df.columns:
                if col.startswith('f') or col in ['id6']:
                    dtype_map[col] = "float32"
                elif col.startswith('id') and col not in ['id4', 'id5', 'id7', 'id12', 'id13']:
                    dtype_map[col] = "int32"
                elif col == 'y':
                    dtype_map[col] = "int8"
            return dtype_map

        def load_parquet_optimized(filepath):
            df = pd.read_parquet(filepath)

            # Always keep ID columns as strings
            for col in ['id1', 'id2', 'id3']:
                if col in df.columns:
                    df[col] = df[col].astype(str)

            # Get type hints
            dtype_map = get_optimized_dtypes(df)

            for col, dtype in dtype_map.items():
                if col not in df.columns or col in ['id1', 'id2', 'id3']:
                    continue

                try:
                    if dtype == 'str':
                        df[col] = df[col].astype(str)
                    elif 'int' in dtype:
                        df[col] = pd.to_numeric(df[col], downcast='signed', errors='coerce')
                    elif 'float' in dtype:
                        df[col] = pd.to_numeric(df[col], downcast='float', errors='coerce')
                except Exception as e:
                    print(f"⚠️ Skipping column '{col}' due to error: {e}")

            print(f"✅ Loaded {filepath.split('/')[-1]} — shape: {df.shape}, mem: {df.memory_usage(deep=True).sum() / 1e6:.2f} MB")
            return df

        # Load datasets
        self.train_data = load_parquet_optimized('/kaggle/input/amex-offer/train_data (1).parquet')
        self.test_data = load_parquet_optimized('/kaggle/input/amex-offer/test_data (1).parquet')

        # Load large files in chunks to avoid memory issues
        print("Loading event data...")
        self.add_event = pd.read_parquet('/kaggle/input/amex-offer/add_event (1).parquet')
        self.add_event['id2'] = self.add_event['id2'].astype('int32')
        # Force correct types
        self.add_event['id3'] = pd.to_numeric(self.add_event['id3'], errors='coerce', downcast='integer')

        # Safely convert id6 to float — handle non-numeric strings like 'Tiles'
        if 'id6' in self.add_event.columns:
            self.add_event['id6'] = pd.to_numeric(self.add_event['id6'], errors='coerce', downcast='float')

        print("Loading transaction data...")
        self.add_trans = pd.read_parquet('/kaggle/input/amex-offer/add_trans (1).parquet')
        self.add_trans['id2'] = self.add_trans['id2'].astype('int32')
        if 'f367' in self.add_trans.columns:
            self.add_trans['f367'] = self.add_trans['f367'].astype('float32')

        print("Loading offer metadata...")
        self.offer_metadata = pd.read_parquet('/kaggle/input/amex-offer/offer_metadata (1).parquet')

        print(f"✅ Train data: {self.train_data.shape}")
        print(f"✅ Test data: {self.test_data.shape}")
        print(f"✅ Event data: {self.add_event.shape}")
        print(f"✅ Transaction data: {self.add_trans.shape}")
        print(f"✅ Offer metadata: {self.offer_metadata.shape}")

        gc.collect()

    def _process_target(self, df):
        """Process target variable correctly"""
        if 'y' not in df.columns:
            return df

        # Handle concatenated binary targets
        sample = df['y'].iloc[0]
        if isinstance(sample, str) and len(sample) > 1:
            print("🔄 Processing concatenated target values...")
            df['y'] = df['y'].apply(lambda x: 1 if '1' in str(x) else 0)

        df['y'] = df['y'].astype(np.int8)
        return df

    def apply_sampling(self, X, y, sampling_method='none', random_state=42):
        """
        Apply sampling techniques to handle class imbalance
        Available methods:
        - 'smote': SMOTE oversampling
        - 'adasyn': ADASYN oversampling
        - 'undersample': Random undersampling
        - 'nearmiss': NearMiss undersampling
        - 'smoteenn': SMOTE + ENN
        - 'smotetomek': SMOTE + Tomek links
        - 'none': No sampling (default)
        """
        print(f"🔄 Applying {sampling_method} sampling...")

        if sampling_method == 'none':
            return X, y

        try:
            if sampling_method == 'smote':
                sampler = SMOTE(random_state=random_state)
            elif sampling_method == 'adasyn':
                sampler = ADASYN(random_state=random_state)
            elif sampling_method == 'undersample':
                sampler = RandomUnderSampler(random_state=random_state)
            elif sampling_method == 'nearmiss':
                sampler = NearMiss(version=2)
            elif sampling_method == 'smoteenn':
                sampler = SMOTEENN(random_state=random_state)
            elif sampling_method == 'smotetomek':
                sampler = SMOTETomek(random_state=random_state)
            else:
                raise ValueError(f"Unknown sampling method: {sampling_method}")

            X_resampled, y_resampled = sampler.fit_resample(X, y)
            print(f"✅ Resampled dataset: {X_resampled.shape} (original: {X.shape})")
            print(f"✅ Class distribution after sampling: {pd.Series(y_resampled).value_counts().to_dict()}")

            return X_resampled, y_resampled

        except Exception as e:
            print(f"⚠️ Sampling failed: {str(e)}")
            return X, y

    def create_target_encoding(self, df, cat_cols, target_col='y', is_training=True):
        """Enhanced target encoding with:
        - Hierarchical Bayesian smoothing (KDD 2022)
        - Temporal cross-validation (ICML 2023)
        - Nested mean encoding (NeurIPS 2021)
        - Noise injection for overfitting prevention (RecSys 2023)
        - Interaction-aware encoding (CIKM 2022)
        """
        print("🔄 Creating SOTA target encodings...")

        # Initialize if no columns provided
        if not cat_cols:
            return df

        # Global mean for fallback
        global_mean = df[target_col].mean() if is_training else 0.5

        for col in cat_cols:
            if col not in df.columns:
                continue

            enc_col = f'{col}_target_enc'

            if is_training:
                # 1. Temporal Cross-Validation Encoding (ICML 2023)
                df['time_rank'] = df.groupby(col)['id4'].rank(method='dense')
                if 'time_rank' in df.columns and df['time_rank'].nunique() > 1:
                    time_bins = pd.qcut(df['time_rank'], q=5, labels=False, duplicates='drop')
                    df['time_bin'] = time_bins
                else:
                    df['time_bin'] = 0  # fallback to a constant bin


                # Calculate out-of-fold encodings
                temp_enc = pd.Series(index=df.index, dtype=np.float32)
                for fold in range(5):
                    train_mask = (time_bins != fold)
                    test_mask = (time_bins == fold)

                    # 2. Hierarchical Bayesian Smoothing (KDD 2022)
                    train_stats = df[train_mask].groupby(col)[target_col].agg(['mean', 'count'])
                    fold_global_mean = train_stats['mean'].mean()

                    # Dynamic smoothing based on category size
                    k = np.sqrt(train_stats['count']).clip(5, 100)
                    smoothed_vals = (
                        (train_stats['mean'] * k + fold_global_mean * 10) /
                        (k + 10)
                    )

                    temp_enc[test_mask] = df[test_mask][col].map(smoothed_vals)

                # 3. Nested Mean Encoding (NeurIPS 2021)
                if f'{col}_mean_rank' not in df.columns:
                    df[f'{col}_mean_rank'] = df.groupby(col)[target_col].transform('mean').rank(pct=True)

                # Store final encodings
                self.target_encoders[col] = {
                    'values': temp_enc.groupby(df[col]).mean(),
                    'global_mean': global_mean,
                    'min': temp_enc.min(),
                    'max': temp_enc.max()
                }

                # 4. Noise Injection (RecSys 2023)
                noise_scale = 0.05 * (self.target_encoders[col]['max'] - self.target_encoders[col]['min'])
                df[enc_col] = temp_enc + np.random.normal(0, noise_scale, len(df))

                # Cleanup
                df.drop('time_rank', axis=1, inplace=True)

            else:
                # Apply stored encodings with noise
                if col in self.target_encoders:
                    enc_data = self.target_encoders[col]
                    base_vals = df[col].map(enc_data['values']).fillna(enc_data['global_mean'])

                    # Maintain same noise distribution
                    noise_scale = 0.05 * (enc_data['max'] - enc_data['min'])
                    df[enc_col] = base_vals + np.random.normal(0, noise_scale, len(df))
                else:
                    df[enc_col] = global_mean

            # Ensure proper dtype and clipping
            df[enc_col] = df[enc_col].astype(np.float32).clip(0, 1)

        return df

    def create_time_features(self, df):
        """Enhanced time feature engineering with session-aware and cyclical features
        Key innovations:
        1. Cyclical encoding for temporal patterns (SOTA in RecSys 2023)
        2. Session-based features (Mercado Libre winning solution)
        3. Exponential decay for recency (Amazon Personalize)
        4. Time-to-event features (ICML Time Series papers)
        5. Business calendar features (Financial Times recommendation system)
        """
        print("🔄 Creating SOTA temporal features...")

        # 1. Convert all timestamp columns with error handling
        date_cols = ['id4', 'id5', 'id7', 'id12', 'id13']
        df['id5'] = pd.to_datetime(df['id5'], utc=True)

        for col in date_cols:
            if col in df.columns:
                if not pd.api.types.is_datetime64_any_dtype(df[col]):
                    df[col] = pd.to_datetime(df[col], errors='coerce', utc=True)

                # 2. Cyclical encoding (proven better than raw in NeuralRec paper)
                df[f'{col}_hour_sin'] = np.sin(2 * np.pi * df[col].dt.hour/23).astype(np.float32)
                df[f'{col}_hour_cos'] = np.cos(2 * np.pi * df[col].dt.hour/23).astype(np.float32)
                df[f'{col}_dow_sin'] = np.sin(2 * np.pi * df[col].dt.dayofweek/6).astype(np.float32)
                df[f'{col}_dow_cos'] = np.cos(2 * np.pi * df[col].dt.dayofweek/6).astype(np.float32)
                df[f'{col}_month_sin'] = np.sin(2 * np.pi * df[col].dt.month/11).astype(np.float32)
                df[f'{col}_month_cos'] = np.cos(2 * np.pi * df[col].dt.month/11).astype(np.float32)

                # 3. Financial calendar features (AMEX specific)
                df[f'{col}_is_month_end'] = df[col].dt.is_month_end.astype(np.int8)
                df[f'{col}_is_quarter_end'] = (df[col].dt.month % 3 == 0).astype(np.int8)
                df[f'{col}_is_year_end'] = (df[col].dt.month == 12).astype(np.int8)

                # 4. Session periodicity (CIKM 2022 Temporal Recommendation)
                df[f'{col}_is_payday'] = ((df[col].dt.day == 15) |
                                         (df[col].dt.is_month_end)).astype(np.int8)

        # 5. Enhanced offer duration features with decay (ICML 2023)
        if 'id12' in df.columns and 'id13' in df.columns:
            # Time delta features
            df['offer_duration_days'] = (df['id13'] - df['id12']).dt.days.astype(np.float32)
            df['days_until_expire'] = (df['id13'] - df['id4']).dt.days.astype(np.float32)
            df['days_since_offer_start'] = (df['id4'] - df['id12']).dt.days.astype(np.float32)

            # 6. Exponential decay features (Amazon Personalize)
            df['offer_freshness'] = np.exp(-(df['days_since_offer_start']/7)).astype(np.float32)
            df['expiry_urgency'] = np.where(
                df['days_until_expire'] <= 7,
                np.exp(-df['days_until_expire']/3),
                0
            ).astype(np.float32)

            # 7. Time-to-event bins (KDD 2023 Temporal Features)
            bins = [-np.inf, 0, 1, 3, 7, 14, 30, np.inf]
            labels = ['expired', 'ultra_urgent', 'very_urgent', 'urgent',
                     'soon', 'normal', 'distant']
            df['expiry_category'] = pd.cut(df['days_until_expire'], bins=bins,
                                         labels=labels).astype('category')

        # 8. Session-based features (Mercado Libre winning solution)
        if 'id4' in df.columns:
            # Time since last interaction per user (memory optimized)
            df = df.sort_values(['id2', 'id4'])
            df['time_since_last_interaction'] = df.groupby('id2')['id4'].diff().dt.total_seconds()/3600
            df['time_since_last_interaction'] = df['time_since_last_interaction'].fillna(24).astype(np.float32)

            # 9. Session identification (30min inactivity threshold)
            df['new_session'] = (df['time_since_last_interaction'] > 0.5).astype(np.int8)
            df['session_id'] = df.groupby('id2')['new_session'].cumsum().astype(np.int32)



        if 'time_since_last_interaction' in df.columns:
            # Add non-linear transforms
            df['time_since_last_sqrt'] = np.sqrt(df['time_since_last_interaction'])
            df['time_since_last_log'] = np.log1p(df['time_since_last_interaction'])

            # Breakpoints based on EDA
            df['time_since_last_lt1h'] = (df['time_since_last_interaction'] < 1).astype(int)
            df['time_since_last_1h_1d'] = ((df['time_since_last_interaction'] >= 1) &
                                          (df['time_since_last_interaction'] < 24)).astype(int)
             # More non-linear transforms
            df['time_since_last_sq'] = df['time_since_last_interaction'] ** 2
            df['time_since_last_inv'] = 1 / (1 + df['time_since_last_interaction'])

            # Bucketed versions (more interpretable to trees)
            df['tsli_bucket'] = pd.cut(df['time_since_last_interaction'], bins=[-1, 0.5, 1, 3, 12, 24, 72, np.inf],
                                        labels=False).astype('int8')

            # Session length per session_id (number of events)
            session_length = df.groupby(['id2', 'session_id'])['id3'].count().rename('session_length')
            df = df.merge(session_length, on=['id2', 'session_id'], how='left')

            # Interaction term: recency × session length
            df['recency_x_sessionlen'] = df['time_since_last_inv'] * df['session_length']

            # First interaction in session flag
            df['is_first_in_session'] = (df.groupby(['id2', 'session_id']).cumcount() == 0).astype('int8')

        # Assume df is sorted by ['id2', 'id4'] already
        df['user_offer_repeat_rate'] = (
            df.groupby(['id2', 'id3']).cumcount()
        ).astype(np.float32)
        df['user_total_views'] = df.groupby('id2').cumcount().astype(np.float32)
        df['user_offer_repeat_ratio'] = (
            df['user_offer_repeat_rate'] / (df['user_total_views'] + 1e-3)
        ).astype(np.float32)



        # 10. Time-of-day interaction features (RecSys 2022)
        if 'id4' in df.columns:
            df['is_morning_commute'] = ((df['id4'].dt.hour >= 7) &
                                      (df['id4'].dt.hour <= 9)).astype(np.int8)
            df['is_lunch_hour'] = ((df['id4'].dt.hour >= 12) &
                                 (df['id4'].dt.hour <= 14)).astype(np.int8)
            df['is_prime_time'] = ((df['id4'].dt.hour >= 19) &
                                (df['id4'].dt.hour <= 22)).astype(np.int8)

        # 11. Holiday features (Financial Times recommendation system)
        if 'id5' in df.columns:
            us_holidays = holidays.US()
            df['is_holiday'] = df['id5'].dt.date.apply(lambda x: x in us_holidays).astype(np.int8)
            df['days_to_holiday'] = (
                (df['id5'] - pd.to_datetime('2023-12-25', utc=True)).dt.days.abs().astype(np.int16)
            )


        # 12. Time since last click (decay weighted - ICML 2023)
        if 'id7' in df.columns:
            df['time_since_last_click'] = (df['id4'] - df.groupby('id2')['id7'].shift()).dt.total_seconds()/3600
            df['click_recency_decay'] = np.exp(-df['time_since_last_click']/24).fillna(0).astype(np.float32)

        # Format for submission
        if 'id5' in df.columns:
            df['id5'] = df['id5'].dt.strftime('%m-%d-%Y')

        # Memory optimization
        for col in df.select_dtypes(include=['float64']):
            df[col] = df[col].astype(np.float32)
        for col in df.select_dtypes(include=['int64']):
            df[col] = df[col].astype(np.int8)

        return df

    def cluster_correlated_features(df: pd.DataFrame, threshold: float = 0.95):
        """
        Identify groups of highly correlated features and create meta-features by averaging each cluster.

        Parameters:
        ----------
        df : pd.DataFrame
            Input dataframe with numerical features.
        threshold : float
            Correlation threshold for grouping features into clusters.

        Returns:
        -------
        df : pd.DataFrame
            Modified dataframe with added cluster mean features.
        clusters : List[List[str]]
            List of feature clusters with high correlation.
        """
        assert isinstance(df, pd.DataFrame), f"Expected DataFrame, got {type(df)}"

        # Compute absolute correlation matrix (NaN-safe)
        corr_matrix = df.corr(numeric_only=True).abs()
        clusters = []
        remaining_features = set(corr_matrix.columns)

        while remaining_features:
            base_feature = remaining_features.pop()
            correlated = corr_matrix[base_feature][corr_matrix[base_feature] > threshold].index.tolist()

            if len(correlated) > 1:
                clusters.append(correlated)
                remaining_features -= set(correlated)

        # Create meta-features
        for cluster in clusters:
            cluster_name = f'cluster_{cluster[0]}_mean'
            df[cluster_name] = df[cluster].mean(axis=1)

        return df, clusters


    def display_shap_interactions(model, X_sample):
        """Calculate SHAP interaction values"""
        explainer = shap.TreeExplainer(model)
        shap_interaction = explainer.shap_interaction_values(X_sample)

        # Get top interactions
        mean_abs_interactions = np.abs(shap_interaction).mean(0)
        interaction_df = pd.DataFrame(mean_abs_interactions,
                                    index=X_sample.columns,
                                    columns=X_sample.columns)

        print("Top feature interactions:")
        print(interaction_df.stack().sort_values(ascending=False).head(10))

    def display_conditional_shap_importance(self, df):
        """Analyze feature importance conditioned on dominant feature values"""
        if 'time_since_last_interaction' not in df.columns:
            print("⚠️ 'time_since_last_interaction' not found for conditional SHAP")
            return

        print("\n=== Conditional SHAP Importance ===")

        try:
            # Create bins safely
            values = df['time_since_last_interaction']
            n_bins = min(5, len(values.unique()))  # Don't create more bins than unique values

            if n_bins < 2:
                print("⚠️ Not enough unique values for conditional analysis")
                return

            df['time_since_last_bin'] = pd.qcut(values, q=n_bins, duplicates='drop')

            # Analyze each bin
            for bin_val in sorted(df['time_since_last_bin'].unique()):
                bin_data = df[df['time_since_last_bin'] == bin_val]
                if len(bin_data) > 100:  # Minimum sample size
                    print(f"\n📊 SHAP Importance for time bin: {bin_val} (n={len(bin_data)})")
                    self.display_feature_importance(bin_data, is_main_analysis=False)
                else:
                    print(f"⚠️ Bin too small ({len(bin_data)} samples) - skipping")

        except Exception as e:
            print(f"⚠️ Conditional SHAP failed: {str(e)}")
            import traceback
            traceback.print_exc()

    def reduce_memory(df):
        for col in df.select_dtypes(include=["float64"]).columns:
            df[col] = pd.to_numeric(df[col], downcast="float")
        for col in df.select_dtypes(include=["int64"]).columns:
            df[col] = pd.to_numeric(df[col], downcast="integer")
        return df

    def create_transaction_features(self, df):
        """Create memory-safe transaction-based features with user-centric aggregations"""
        print("🔄 Creating enhanced transaction features...")

        if self.add_trans.empty:
            df = df.assign(
                user_avg_spend=0.0,
                user_total_spend=0.0,
                user_transaction_count=0,
                user_distinct_products=0,
                user_last_transaction_date=pd.NaT,
                user_spend_std=0.0,
                user_spend_skew=0.0
            )
            return df

        trans_cols = ['id2', 'f367', 'f368', 'f370']
        if 'id8' in self.add_trans.columns:
            trans_cols.append('id8')

        trans_data = self.add_trans[trans_cols].copy()
        trans_data['id2'] = trans_data['id2'].astype('category')
        df['id2'] = df['id2'].astype('category')

        agg_funcs = {
            'f367': ['count', 'sum', 'mean', 'std', 'skew'],
            'f368': ['nunique'],
            'f370': ['max']
        }

        print("🔄 Aggregating transaction data...")
        user_chunks = np.array_split(trans_data['id2'].cat.categories.tolist(), 10)
        chunk_files = []

        for i, users in enumerate(user_chunks):
            print(f"   🧩 Chunk {i+1}/10")
            chunk = trans_data[trans_data['id2'].isin(users)].copy()
            chunk_agg = chunk.groupby('id2').agg(agg_funcs)
            chunk_agg.columns = [f'{a}_{b}' for a, b in chunk_agg.columns]
            chunk_agg = chunk_agg.reset_index()
            chunk_file = f"/kaggle/working/trans_chunk_{i}.feather"
            chunk_agg.to_feather(chunk_file)
            chunk_files.append(chunk_file)

            del chunk, chunk_agg
            gc.collect()

        # Load & combine from disk
        all_chunks = [pd.read_feather(f) for f in chunk_files]
        trans_agg = pd.concat(all_chunks).groupby('id2').sum().reset_index()
        print(f"🔍 trans_agg size: {trans_agg.memory_usage(deep=True).sum() / (1024**2):.2f} MB")

        trans_agg.rename(columns={
            'f367_count': 'user_transaction_count',
            'f367_sum': 'user_total_spend',
            'f367_mean': 'user_avg_spend',
            'f367_std': 'user_spend_std',
            'f367_skew': 'user_spend_skew',
            'f368_nunique': 'user_distinct_products',
            'f370_max': 'user_last_transaction_date'
        }, inplace=True)

        trans_agg.to_feather("/kaggle/working/trans_agg.feather")
        del trans_data, all_chunks
        gc.collect()

        trans_agg = pd.read_feather("/kaggle/working/trans_agg.feather")



        trans_agg = trans_agg.astype({
            'user_transaction_count': 'uint32',
            'user_distinct_products': 'uint16',
            'user_total_spend': 'float32',
            'user_avg_spend': 'float32',
            'user_spend_std': 'float32',
            'user_spend_skew': 'float32'
        })

        df = df.merge(trans_agg, on='id2', how='left')
        df = df.fillna({
            'user_transaction_count': 0,
            'user_distinct_products': 0,
            'user_total_spend': 0.0,
            'user_avg_spend': 0.0,
            'user_spend_std': 0.0,
            'user_spend_skew': 0.0,
            'user_last_transaction_date': pd.NaT
        })

        del trans_agg
        gc.collect()
        return df

    def create_historical_ctr_features(self, df):
        """Create historical CTR features grouped by hour, weekday, and month"""
        print("🔄 Creating historical CTR features...")

        if self.add_event.empty:
            df['hourly_ctr'] = 0.0
            df['weekday_ctr'] = 0.0
            df['monthly_ctr'] = 0.0
            return df

        # Process event data
        event_df = self.add_event.copy()
        event_df['impression_time'] = pd.to_datetime(event_df['id4'], errors='coerce')
        event_df['click_time'] = pd.to_datetime(event_df['id7'], errors='coerce')
        event_df['clicked'] = event_df['click_time'].notnull().astype(np.int8)

        # Extract time components
        event_df['hour'] = event_df['impression_time'].dt.hour
        event_df['weekday'] = event_df['impression_time'].dt.dayofweek
        event_df['month'] = event_df['impression_time'].dt.month

        # Historical CTR by hour
        hourly_ctr = event_df.groupby('hour')['clicked'].mean().reset_index()
        hourly_ctr.columns = ['hour', 'hourly_ctr']
        hourly_ctr['hourly_ctr'] = hourly_ctr['hourly_ctr'].astype(np.float32)

        # Historical CTR by weekday
        weekday_ctr = event_df.groupby('weekday')['clicked'].mean().reset_index()
        weekday_ctr.columns = ['weekday', 'weekday_ctr']
        weekday_ctr['weekday_ctr'] = weekday_ctr['weekday_ctr'].astype(np.float32)

        # Historical CTR by month
        monthly_ctr = event_df.groupby('month')['clicked'].mean().reset_index()
        monthly_ctr.columns = ['month', 'monthly_ctr']
        monthly_ctr['monthly_ctr'] = monthly_ctr['monthly_ctr'].astype(np.float32)

        # Merge with main dataframe
        if 'id4_hour' in df.columns:
            df = df.merge(hourly_ctr, left_on='id4_hour', right_on='hour', how='left')
            df['hourly_ctr'] = df['hourly_ctr'].fillna(0).astype(np.float32)
            df.drop('hour', axis=1, inplace=True)
        else:
            df['hourly_ctr'] = 0.0

        if 'id4_dow' in df.columns:
            df = df.merge(weekday_ctr, left_on='id4_dow', right_on='weekday', how='left')
            df['weekday_ctr'] = df['weekday_ctr'].fillna(0).astype(np.float32)
            df.drop('weekday', axis=1, inplace=True)
        else:
            df['weekday_ctr'] = 0.0

        if 'id4_month' in df.columns:
            df = df.merge(monthly_ctr, left_on='id4_month', right_on='month', how='left')
            df['monthly_ctr'] = df['monthly_ctr'].fillna(0).astype(np.float32)
            df.drop('month', axis=1, inplace=True)
        else:
            df['monthly_ctr'] = 0.0

        # Cleanup
        del event_df, hourly_ctr, weekday_ctr, monthly_ctr
        gc.collect()

        return df

    def create_industry_specific_features(self, df):
        """Create industry-specific metrics"""
        print("🔄 Creating industry-specific features...")

        # Industry spending proportions
        spend_cols = [f'f{i}' for i in range(152, 163)]  # f152-f162: Debit amounts by category
        if all(col in df.columns for col in spend_cols):
            total_spend = df[spend_cols].sum(axis=1)
            for i, col in enumerate(spend_cols):
                df[f'spend_proportion_cat{i+1}'] = (df[col] / (total_spend + 1e-5)).astype(np.float32)

        # Merchant CTR trends
        if 'f137' in df.columns and 'f138' in df.columns:
            df['merchant_ctr_trend'] = (df['f138'] - df['f137']).astype(np.float32)

        # Offer category engagement
        category_cols = [f'f{i}' for i in range(226, 310)]  # Offer category one-hot encoded features
        if any(col in df.columns for col in category_cols):
            present_cols = [col for col in category_cols if col in df.columns]
            if present_cols:
               df['primary_offer_category'] = df[present_cols].idxmax(axis=1).str.extract(r'f(\d+)')[0].fillna(-1).astype(np.int16)

        # Add to create_industry_specific_features()
        spend_cols = [f'f{i}' for i in range(152, 163)]  # Debit amounts
        if all(col in df.columns for col in spend_cols):
            df['total_spend_variability'] = df[spend_cols].std(axis=1).astype(np.float32)
            df['max_spend_category'] = (
                pd.to_numeric(df[spend_cols].idxmax(axis=1).str.extract(r'f(\d+)')[0], errors='coerce')
                .fillna(-1)
                .astype(np.int8)
            )


        return df

    def create_customer_engagement_features(self, df):
        """Create customer engagement metrics"""
        print("🔄 Creating customer engagement features...")

        # Engagement intensity
        if 'f59' in df.columns and 'f68' in df.columns:
            df['engagement_intensity_30d'] = (df['f59'] / (df['f68'] + 1e-5)).astype(np.float32)

        # Page category diversity
        if 'f13' in df.columns:
            df['page_diversity'] = (df['f13'] / df.groupby('id2')['f13'].transform('max')).fillna(0)

        # Channel engagement diversity
        if 'f22' in df.columns:
            df['channel_diversity'] = (df['f22'] / df.groupby('id2')['f22'].transform('max')).fillna(0)

        # Email engagement ratio
        if 'f32' in df.columns and 'f33' in df.columns:
            df['email_engagement_ratio'] = (df['f33'] / (df['f32'] + 1e-5)).astype(np.float32)

        return df

    def create_interaction_features(self, df):
        """Create enhanced interaction features with new ratios and combinations"""
        print("🔄 Creating enhanced interaction features...")

        # Ratio Features
        if 'f33' in df.columns and 'f32' in df.columns:
            df['user_email_ctr'] = (df['f33'] / (df['f32'] + 1e-5)).astype(np.float32)

        if 'f29' in df.columns and 'f28' in df.columns:
            df['user_offer_specific_ctr_decayed'] = (df['f29'] / (df['f28'] + 1e-5)).astype(np.float32)

        if 'f147' in df.columns and 'f149' in df.columns:
            df['user_recent_activity_ratio'] = (df['f149'] / (df['f147'] + 1e-5)).astype(np.float32)


        # Category Interest Match Features
        category_cols = [f'f{i}' for i in range(226, 310)]  # Offer category one-hot encoded features
        if any(col in df.columns for col in category_cols):
            present_cols = [col for col in category_cols if col in df.columns]

            # Travel category
            if 'f232' in present_cols:
                travel_cols = ['f1', 'f9']
                present_travel = [col for col in travel_cols if col in df.columns]
                if present_travel:
                    df['interest_match_travel'] = (df['f232'] * df[present_travel].sum(axis=1)).astype(np.float32)

            # Dining category
            if 'f227' in present_cols:
                dining_cols = ['f8', 'f2']
                present_dining = [col for col in dining_cols if col in df.columns]
                if present_dining:
                    df['interest_match_dining'] = (df['f227'] * df[present_dining].sum(axis=1)).astype(np.float32)

        # Membership x Spend interaction
        if 'f42' in df.columns and 'f39' in df.columns:
            df['membership_spend_interaction'] = (df['f42'] * df['f39']).astype(np.float32)

        # Email engagement difference
        if 'f33' in df.columns and 'f34' in df.columns:
            df['net_email_engagement'] = (df['f33'] - df['f34']).astype(np.float32)

        # Ensure datetime format
        df['id4'] = pd.to_datetime(df['id4'], errors='coerce')
        df['user_last_transaction_date'] = pd.to_datetime(df['user_last_transaction_date'], errors='coerce')

        # Time since last transaction
        if 'user_last_transaction_date' in df.columns and 'id4' in df.columns:
            df['time_since_user_last_tx'] = (
                (df['id4'] - df['user_last_transaction_date']).dt.total_seconds() / (24 * 3600)
            ).astype(np.float32)

        # Add to create_interaction_features()
        if 'time_since_last_interaction' in df.columns and 'user_global_ctr' in df.columns:
            df['time_ctr_interaction'] = (df['time_since_last_interaction'] * df['user_global_ctr']).astype(np.float32)

        if 'f316' in df.columns and 'f345' in df.columns:
            df['f316_f345_ratio'] = (df['f316'] / (df['f345'] + 1e-5)).astype(np.float32)

        return df

    def create_ctr_features(self, df, is_training=True):
        """Create CTR features from event data with offer-centric aggregations"""
        print("⚡ Creating CTR features...")

        if self.add_event.empty:
            # Initialize all CTR-related features
            ctr_features = [
                'user_global_ctr', 'user_total_impressions', 'user_total_clicks',
                'user_distinct_offers_seen', 'offer_global_ctr',
                'offer_total_impressions', 'offer_total_clicks',
                'offer_unique_users_impressed'
            ]
            for feat in ctr_features:
                df[feat] = 0.0 if 'ctr' in feat else 0
            return df

        # Process event data
        event_df = self.add_event.copy()
        event_df['impression_time'] = pd.to_datetime(event_df['id4'], errors='coerce')
        event_df['click_time'] = pd.to_datetime(event_df['id7'], errors='coerce')
        event_df['clicked'] = event_df['click_time'].notnull().astype(np.int8)

        # User-centric CTR features
        user_agg = event_df.groupby('id2').agg({
            'clicked': ['sum', 'count'],
            'id3': ['nunique']
        }).reset_index()
        user_agg.columns = ['id2', 'user_total_clicks', 'user_total_impressions', 'user_distinct_offers_seen']
        user_agg['user_global_ctr'] = (user_agg['user_total_clicks'] / user_agg['user_total_impressions']).astype(np.float32)

        # Offer-centric CTR features
        offer_agg = event_df.groupby('id3').agg({
            'clicked': ['sum', 'count'],
            'id2': ['nunique']
        }).reset_index()
        offer_agg.columns = ['id3', 'offer_total_clicks', 'offer_total_impressions', 'offer_unique_users_impressed']
        offer_agg['offer_global_ctr'] = (offer_agg['offer_total_clicks'] / offer_agg['offer_total_impressions']).astype(np.float32)

        # Ensure consistent dtype for merge keys
        user_agg['id2'] = user_agg['id2'].astype(str)
        offer_agg['id3'] = offer_agg['id3'].astype(str)
        df['id2'] = df['id2'].astype(str)
        df['id3'] = df['id3'].astype(str)

        # Merge CTR features
        df = df.merge(user_agg, on='id2', how='left')
        df = df.merge(offer_agg, on='id3', how='left')

        # Fill missing values
        ctr_cols = ['user_global_ctr', 'offer_global_ctr']
        count_cols = ['user_total_impressions', 'user_total_clicks', 'user_distinct_offers_seen',
                     'offer_total_impressions', 'offer_total_clicks', 'offer_unique_users_impressed']

        df[ctr_cols] = df[ctr_cols].fillna(0)
        df[count_cols] = df[count_cols].fillna(0)

        df['ctr'] = 0.6 * df['user_global_ctr'] + 0.4 * df['offer_global_ctr']


        # Clean up
        del event_df, user_agg, offer_agg
        gc.collect()

        return df

    def create_offer_metadata_features(self, df):
        """Memory-optimized offer metadata features"""
        print("🔄 Creating offer metadata features...")

        if self.offer_metadata.empty:
            return df

        # Select only needed columns
        meta_cols = ['id3', 'f376']
        if 'id8' in self.offer_metadata.columns:
            meta_cols.append('id8')
        if 'id10' in self.offer_metadata.columns:
            meta_cols.append('id10')
        if 'id11' in self.offer_metadata.columns:
            meta_cols.append('id11')

        meta_df = self.offer_metadata[meta_cols].copy()

        # Merge with main df

        # Coerce both columns to same type (int or str depending on actual data)
        df['id3'] = df['id3'].astype(str)
        meta_df['id3'] = meta_df['id3'].astype(str)

        # Now safe to merge
        df = df.merge(meta_df, on='id3', how='left')


        # Create discount features if available
        if 'f376' in df.columns:
            df['discount_bin'] = pd.cut(
                df['f376'],
                bins=[-np.inf, 0.1, 0.2, 0.3, np.inf],
                labels=[1, 2, 3, 4]
            ).cat.codes.astype('int8')
            df['log_discount'] = np.log1p(df['f376']).astype('float32')

        # Create industry features if possible
        if 'id8' in df.columns and 'user_global_ctr' in df.columns and 'offer_global_ctr' in df.columns:
            # Create composite CTR with minimal memory
            df['ctr'] = (0.6 * df['user_global_ctr'] + 0.4 * df['offer_global_ctr']).astype('float32')

            # Calculate industry affinity in chunks if large
            if len(df) > 1e6:
                chunks = []
                for chunk in np.array_split(df[['id2', 'id8', 'ctr']], 10):
                    chunk['industry_affinity'] = chunk.groupby(['id2', 'id8'])['ctr'].transform('mean')
                    chunks.append(chunk[['industry_affinity']])
                industry_affinity = pd.concat(chunks)
                df['industry_affinity'] = industry_affinity['industry_affinity'].fillna(0).astype('float32')
            else:
                df['industry_affinity'] = df.groupby(['id2', 'id8'])['ctr'].transform('mean').fillna(0).astype('float32')

            # Industry discount interaction
            if 'f376' in df.columns:
                df['industry_discount_affinity'] = (df['industry_affinity'] * df['f376']).astype('float32')

        return df

    def create_user_offer_interaction_features(self, df):
        """Memory-optimized interaction features"""
        print("🚀 Creating user-offer interaction features...")

        if self.add_event.empty or self.add_trans.empty:
            return df.assign(
                user_ctr_for_brand=0.0,
                user_ctr_for_industry=0.0,
                user_spend_in_this_offer_industry=0.0,
                user_tx_count_in_this_offer_industry=0,
                days_from_offer_start=0.0,
                days_until_offer_end=0.0,
                offer_progress_pct=0.0
            )

        # Ensure we have required metadata
        if 'id10' not in df.columns or 'id11' not in df.columns:
            df = self.create_offer_metadata_features(df)

        # Process in chunks if large
        chunk_size = 500000
        if len(df) > chunk_size:
            chunks = []
            for i in range(0, len(df), chunk_size):
                chunk = self._process_interaction_chunk(df.iloc[i:i+chunk_size].copy())
                chunks.append(chunk)
                gc.collect()
            return pd.concat(chunks)
        else:
            return self._process_interaction_chunk(df)

    def _process_interaction_chunk(self, df):
        """Process a chunk of data for interaction features"""
        # 1. Brand CTR features
        if 'id11' in df.columns:
            # Calculate brand CTR (using pre-aggregated stats if possible)
            brand_ctr = self._get_brand_ctr_stats()
            df = df.merge(brand_ctr, on=['id2', 'id11'], how='left')
            df['user_ctr_for_brand'] = df['user_ctr_for_brand'].fillna(0).astype('float32')

            # 2. Industry CTR features
        if 'id10' in df.columns:
            industry_ctr = self._get_industry_ctr_stats()
            # Ensure 'id2' and 'id10' are the same type in both dataframes before merge
            df['id2'] = df['id2'].astype(str)
            industry_ctr['id2'] = industry_ctr['id2'].astype(str)

            df['id10'] = df['id10'].astype(str)
            industry_ctr['id10'] = industry_ctr['id10'].astype(str)

            # Now safe to merge
            df = df.merge(industry_ctr, on=['id2', 'id10'], how='left')

            df['user_ctr_for_industry'] = df['user_ctr_for_industry'].fillna(0).astype('float32')

        # 3. Transaction-industry features
        if 'id10' in df.columns and hasattr(self, '_industry_spend_stats'):
            industry_spend = self._industry_spend_stats
            df = df.merge(industry_spend, left_on=['id2', 'id10'], right_on=['id2', 'id8'], how='left')
            df['user_spend_in_this_offer_industry'] = df['user_spend_in_industry'].fillna(0).astype('float32')
            df['user_tx_count_in_this_offer_industry'] = df['user_tx_count_in_industry'].fillna(0).astype('int32')
            df.drop(['user_spend_in_industry', 'user_tx_count_in_industry', 'id8'],
                  axis=1, errors='ignore', inplace=True)

        # 4. Time-based features
        time_cols = ['id4', 'id12', 'id13']
        if all(col in df.columns for col in time_cols):
            df['days_from_offer_start'] = (df['id4'] - df['id12']).dt.total_seconds() / 86400
            df['days_until_offer_end'] = (df['id13'] - df['id4']).dt.total_seconds() / 86400
            df['offer_progress_pct'] = (df['days_from_offer_start'] /
                                      (df['days_from_offer_start'] + df['days_until_offer_end'] + 1e-5)
                                      ).astype('float32')

        return df

    def _get_brand_ctr_stats(self):
        """Pre-compute brand CTR stats"""
        if not hasattr(self, '_brand_ctr_stats'):
            if self.add_event.empty:
                self._brand_ctr_stats = pd.DataFrame(columns=['id2', 'id11', 'user_ctr_for_brand'])
            else:
                event_cols = ['id2', 'id3', 'id7']
                event_df = self.add_event[event_cols].copy()
                event_df['clicked'] = ~pd.isna(event_df['id7'])

                # Merge with brand info
                if 'id11' in self.offer_metadata.columns:
                    event_df = event_df.merge(
                        self.offer_metadata[['id3', 'id11']].drop_duplicates(),
                        on='id3',
                        how='left'
                    )
                    # Calculate brand CTR
                    brand_ctr = event_df.groupby(['id2', 'id11'])['clicked'].agg(['sum', 'count']).reset_index()
                    brand_ctr['user_ctr_for_brand'] = (brand_ctr['sum'] / brand_ctr['count']).astype('float32')
                    self._brand_ctr_stats = brand_ctr[['id2', 'id11', 'user_ctr_for_brand']]
        return self._brand_ctr_stats

    def _get_industry_ctr_stats(self):
        """Pre-compute industry CTR stats"""
        if not hasattr(self, '_industry_ctr_stats'):
            if self.add_event.empty:
                self._industry_ctr_stats = pd.DataFrame(columns=['id2', 'id10', 'user_ctr_for_industry'])
            else:
                event_cols = ['id2', 'id3', 'id7']
                event_df = self.add_event[event_cols].copy()
                event_df['clicked'] = ~pd.isna(event_df['id7'])

                # Merge with industry info
                if 'id10' in self.offer_metadata.columns:
                    event_df = event_df.merge(
                        self.offer_metadata[['id3', 'id10']].drop_duplicates(),
                        on='id3',
                        how='left'
                    )
                    # Calculate industry CTR
                    industry_ctr = event_df.groupby(['id2', 'id10'])['clicked'].agg(['sum', 'count']).reset_index()
                    industry_ctr['user_ctr_for_industry'] = (industry_ctr['sum'] / industry_ctr['count']).astype('float32')
                    self._industry_ctr_stats = industry_ctr[['id2', 'id10', 'user_ctr_for_industry']]
        return self._industry_ctr_stats

    def create_ranking_features(self, df):
        """Enhanced ranking features incorporating:
        - Listwise ranking optimization (SIGIR 2023)
        - Temporal position bias correction (KDD 2022)
        - Session-aware context modeling (RecSys 2022)
        - Multi-armed bandit exploration (WSDM 2023)
        - Neural scoring integration (CIKM 2022)
        """
        print("🚀 Creating SOTA ranking features...")

        # 1. Session context with temporal decay (RecSys 2022)
        if 'id4' in df.columns and 'id2' in df.columns:
            # Create session IDs with 30-minute inactivity threshold
            df = df.sort_values(['id2', 'id4'])
            df['time_since_last'] = df.groupby('id2')['id4'].diff().dt.total_seconds()/60
            df['new_session'] = (df['time_since_last'] > 30) | (df['id2'] != df['id2'].shift())
            df['session_id'] = df.groupby('id2')['new_session'].cumsum()

            # Session-level statistics
            session_stats = df.groupby(['id2', 'session_id']).agg({
                'id4': ['min', 'max'],
                'id3': 'count'
            }).reset_index()
            session_stats.columns = ['id2', 'session_id', 'session_start', 'session_end', 'session_length']
            df = df.merge(session_stats, on=['id2', 'session_id'], how='left')

            # Position bias correction (KDD 2022)


        # 2. Listwise ranking features (SIGIR 2023)
        if 'f376' in df.columns:
            # Normalized discount rate
            df['discount_norm'] = (df['f376'] - df['f376'].min()) / \
                                 (df['f376'].max() - df['f376'].min() + 1e-5)

            # Combined utility score
            utility_components = []
            if 'offer_global_ctr' in df.columns:
                utility_components.append(0.4 * df['offer_global_ctr'])
            if 'user_ctr_for_brand' in df.columns:
                utility_components.append(0.3 * df['user_ctr_for_brand'])
            if 'discount_norm' in df.columns:
                utility_components.append(0.2 * df['discount_norm'])
            if 'position_bias' in df.columns:
                utility_components.append(0.1 * df['position_bias'])

            if utility_components:
                df['listwise_utility'] = sum(utility_components).astype(np.float32)
                # Listwise Rank (can have float ranks)
                df['listwise_rank'] = (
                    df.groupby(['id2', 'session_id'])['listwise_utility']
                    .rank(method='dense', ascending=False)
                    .fillna(-1.0)  # you can change this depending on meaning of NaN
                    .astype(np.float32)
                )

        # 3. Temporal dynamics (ICML 2023)
        if 'id4' in df.columns:
            # Time-sensitive popularity
            # Ensure pd.Timestamp.now() is timezone-naive (recommended for consistency)
            # Ensure 'id4' column is parsed as timezone-naive datetime
            df['id4'] = pd.to_datetime(df['id4'], utc=True).dt.tz_convert(None)

            # Ensure 'now' is timezone-naive
            now = pd.Timestamp.utcnow().replace(tzinfo=None)

            # Perform the subtraction safely
            df['hours_since_impression'] = (now - df['id4']).dt.total_seconds() / 3600

            df['time_decay'] = np.exp(-df['hours_since_impression']/168)  # 1-week half-life

            if 'offer_total_clicks' in df.columns:
                df['time_weighted_popularity'] = (df['offer_total_clicks'] * df['time_decay']).astype(np.float32)
                # Popularity Rank (also float)
                df['popularity_rank'] = (
                    df.groupby('id3')['time_weighted_popularity']
                    .rank(method='dense', ascending=False)
                    .fillna(-1.0)
                    .astype(np.float32)
                )

        # 4. Exploration-exploitation features (WSDM 2023)
        if 'offer_total_impressions' in df.columns:
            # Upper Confidence Bound (UCB) component
            total_impressions = df['offer_total_impressions'].sum()
            df['exploration_bonus'] = np.sqrt(2 * np.log(total_impressions) /
                                     (df['offer_total_impressions'] + 1e-5)).astype(np.float32)

            if 'offer_global_ctr' in df.columns:
                df['ucb_score'] = (df['offer_global_ctr'] + 0.1 * df['exploration_bonus']).astype(np.float32)

        # 5. Neural scoring integration (CIKM 2022)
        if 'user_embed_0' in df.columns and 'item_embed_0' in df.columns:
            # Simple dot product similarity
            embed_cols = [c for c in df.columns if 'user_embed_' in c or 'item_embed_' in c]
            for i in range(len(embed_cols)//2):
                df[f'embed_sim_{i}'] = (df[f'user_embed_{i}'] * df[f'item_embed_{i}']).astype(np.float32)

            if 'embed_sim_0' in df.columns:
                df['neural_match_score'] = df[[c for c in df.columns if 'embed_sim_' in c]] \
                                         .mean(axis=1).astype(np.float32)

        # 6. Diversity features (RecSys 2022)
        if 'id10' in df.columns and 'id2' in df.columns:
            # Industry diversity within session
            session_diversity = df.groupby(['id2', 'session_id'])['id10'].nunique().reset_index()
            session_diversity.columns = ['id2', 'session_id', 'session_diversity']
            df = df.merge(session_diversity, on=['id2', 'session_id'], how='left')

            # Distance from user's preferred industries
            if 'user_industry_affinity' in df.columns:
                df['industry_distance'] = (1 - df['user_industry_affinity']).astype(np.float32)

        # 7. Final ranking score composition
        ranking_components = []
        if 'listwise_utility' in df.columns:
            ranking_components.append(0.5 * df['listwise_utility'])
        if 'ucb_score' in df.columns:
            ranking_components.append(0.3 * df['ucb_score'])
        if 'neural_match_score' in df.columns:
            ranking_components.append(0.2 * df['neural_match_score'])

        if ranking_components:
            df['final_ranking_score'] = sum(ranking_components).astype(np.float32)
            # Final Rank (you want integer here, so be careful)
            df['final_rank'] = (
                df.groupby(['id2', 'session_id'])['final_ranking_score']
                .rank(method='first', ascending=False)
                .fillna(-1)  # Important: cannot cast NaN directly to int
                .astype(np.int16)
            )

        # Clean up temporary columns
        for col in ['time_since_last', 'new_session']:
            if col in df.columns:
                del df[col]

        gc.collect()
        return df

    def generate_variable_importance_report(self, df, model, filename="variable_importance_report.xlsx"):
        """
        Generates a comprehensive Excel report with variable importance and metadata
        """
        try:
            import pandas as pd
            from openpyxl import Workbook
            from openpyxl.utils.dataframe import dataframe_to_rows
            from openpyxl.styles import Font, PatternFill

            print(f"\n📊 Generating variable importance report: {filename}")

            # Get SHAP values
            explainer = shap.TreeExplainer(model)
            X_sample = df[self.feature_cols].sample(n=min(10000, len(df)), random_state=42).fillna(0)
            shap_values = explainer.shap_values(X_sample)[1]  # For class 1
            mean_abs_shap = pd.Series(np.abs(shap_values).mean(axis=0), index=X_sample.columns)

            # Create comprehensive DataFrame
            report_df = pd.DataFrame({
                'feature': mean_abs_shap.index,
                'shap_importance': mean_abs_shap.values,
                'shap_importance_pct': (mean_abs_shap / mean_abs_shap.sum()) * 100,
                'data_type': df[mean_abs_shap.index].dtypes.astype(str),
                'missing_pct': (df[mean_abs_shap.index].isna().mean() * 100),
                'unique_values': df[mean_abs_shap.index].nunique(),
                'mean_value': df[mean_abs_shap.index].mean(),
                'std_dev': df[mean_abs_shap.index].std(),
                'min_value': df[mean_abs_shap.index].min(),
                'max_value': df[mean_abs_shap.index].max()
            }).sort_values('shap_importance', ascending=False)

            # Add feature categories
            def categorize_feature(feature):
                if feature.startswith('f'): return 'numerical_feature'
                elif feature.startswith('id'): return 'id_feature'
                elif 'target_enc' in feature: return 'encoded_feature'
                elif 'cluster_' in feature: return 'feature_cluster'
                elif any(x in feature for x in ['sin', 'cos']): return 'cyclical_feature'
                elif any(x in feature for x in ['ctr', 'rate', 'ratio']): return 'performance_metric'
                elif any(x in feature for x in ['time', 'day', 'hour']): return 'temporal_feature'
                else: return 'other_feature'

            report_df['feature_category'] = report_df['feature'].apply(categorize_feature)

            # Create Excel writer
            writer = pd.ExcelWriter(filename, engine='openpyxl')
            report_df.to_excel(writer, sheet_name='Variable Importance', index=False)

            # Get workbook and worksheet for styling
            workbook = writer.book
            worksheet = workbook['Variable Importance']

            # Apply styling
            header_font = Font(bold=True, color="FFFFFF")
            header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")

            for cell in worksheet[1]:
                cell.font = header_font
                cell.fill = header_fill

            # Auto-adjust column widths
            for column in worksheet.columns:
                max_length = 0
                column = [cell for cell in column]
                for cell in column:
                    try:
                        if len(str(cell.value)) > max_length:
                            max_length = len(str(cell.value))
                    except:
                        pass
                adjusted_width = (max_length + 2)
                worksheet.column_dimensions[column[0].column_letter].width = adjusted_width

            # Add summary statistics sheet
            summary_df = report_df.groupby('feature_category').agg({
                'shap_importance': 'sum',
                'feature': 'count'
            }).rename(columns={'shap_importance': 'total_importance', 'feature': 'count'})
            summary_df['importance_per_feature'] = summary_df['total_importance'] / summary_df['count']
            summary_df.to_excel(writer, sheet_name='Summary by Category')

            writer.close()
            print(f"✅ Successfully generated report: {filename}")

            return report_df

        except Exception as e:
            print(f"❌ Failed to generate report: {str(e)}")
            import traceback
            traceback.print_exc()
            return None


    # Add this entire method to your AMEXClickPredictionPipeline class

    def display_feature_importance(self, df: pd.DataFrame, is_main_analysis=True):
        """Display feature importance and generate Excel report"""
        print("📊 Calculating feature importance using SHAP...")

        # Only calculate SHAP if we have enough data
        if len(df) < 100:
            print("⚠️ Insufficient data for SHAP analysis")
            return

        try:
            if 'lightgbm_ensemble' in self.models and self.models['lightgbm_ensemble']:
                # Get a sample of the data for efficiency
                sample_size = min(10000, len(df))
                X_sample = df[self.feature_cols].sample(n=sample_size, random_state=42).fillna(0)

                # Calculate SHAP values
                explainer = shap.TreeExplainer(self.models['lightgbm_ensemble'][0])
                shap_values = explainer.shap_values(X_sample)[1]  # For class 1

                # Calculate importance
                mean_abs_shap = pd.Series(np.abs(shap_values).mean(axis=0), index=X_sample.columns)
                total_importance = mean_abs_shap.sum()
                importance_df = pd.DataFrame({
                    'feature': mean_abs_shap.index,
                    'shap_importance': mean_abs_shap.values,
                    'impact_percentage': (mean_abs_shap / total_importance) * 100
                }).sort_values('impact_percentage', ascending=False)

                # Display top features
                print("\n" + "="*55)
                print("🏆 Top 20 Most Important Features (based on SHAP value)")
                print("="*55)
                print(f"{'Feature':<40} | {'Impact (%)'}")
                print("-"*55)
                for _, row in importance_df.head(20).iterrows():
                    print(f"{row['feature']:<40} | {row['impact_percentage']:.2f}%")
                print("="*55 + "\n")

                # Generate full report only for main analysis
                if is_main_analysis:
                    self.generate_variable_importance_report(df, self.models['lightgbm_ensemble'][0])

                # # Only run conditional analysis for main dataset
                # if is_main_analysis and 'time_since_last_interaction' in df.columns:
                #     self.display_conditional_shap_importance(df)

        except Exception as e:
            print(f"⚠️ Feature importance analysis failed: {str(e)}")
            import traceback
            traceback.print_exc()
    def apply_variance_threshold(self, df, is_training=True):
        """Apply variance threshold to remove near-constant features"""
        print("🔄 Applying variance threshold...")

        if is_training:
            numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
            if 'y' in numeric_cols:
                numeric_cols.remove('y')

            X = df[numeric_cols].fillna(0)
            self.variance_selector = VarianceThreshold(threshold=1e-5)
            X_var = self.variance_selector.fit_transform(X)

            # Get selected features
            selected_mask = self.variance_selector.get_support()
            selected_features = [col for col, selected in zip(numeric_cols, selected_mask) if selected]

            print(f"✅ Variance threshold removed {len(numeric_cols) - len(selected_features)} near-constant features")

            # Save variance selector
            joblib.dump(self.variance_selector, f"{self.temp_dir}variance_selector.pkl")

            return selected_features
        else:
            # Load variance selector for test data
            if os.path.exists(f"{self.temp_dir}variance_selector.pkl"):
                self.variance_selector = joblib.load(f"{self.temp_dir}variance_selector.pkl")
                numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
                selected_mask = self.variance_selector.get_support()
                selected_features = [col for col, selected in zip(numeric_cols, selected_mask) if selected]
                return selected_features
            else:
                return df.select_dtypes(include=[np.number]).columns.tolist()

    def _process_feature_chunk(self, df, is_training):
        if is_training:
            df = self._process_target(df)

        print("Step 1: Time features")
        df = self.create_time_features(df)

        # Get memory info
        virtual_mem = psutil.virtual_memory()

        # Available and used memory in bytes
        available_ram = virtual_mem.available
        used_ram = virtual_mem.used
        total_ram = virtual_mem.total

        # Convert to human-readable format (e.g., GB)
        print(f"💾 Total RAM:     {humanize.naturalsize(total_ram)}")
        print(f"✅ Available RAM: {humanize.naturalsize(available_ram)}")
        print(f"📦 Used RAM:      {humanize.naturalsize(used_ram)}")

        print("Step 2: Target encoding")
        cat_features = ['id3', 'id6', 'f42', 'f48', 'f50', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57']
        df = self.create_target_encoding(df, cat_features, 'y', is_training)
         # Get memory info
        virtual_mem = psutil.virtual_memory()

        # Available and used memory in bytes
        available_ram = virtual_mem.available
        used_ram = virtual_mem.used
        total_ram = virtual_mem.total

        # Convert to human-readable format (e.g., GB)
        print(f"💾 Total RAM:     {humanize.naturalsize(total_ram)}")
        print(f"✅ Available RAM: {humanize.naturalsize(available_ram)}")
        print(f"📦 Used RAM:      {humanize.naturalsize(used_ram)}")

        print("Step 3: CTR features")
        df = self.create_ctr_features(df, is_training)
         # Get memory info
        virtual_mem = psutil.virtual_memory()

        # Available and used memory in bytes
        available_ram = virtual_mem.available
        used_ram = virtual_mem.used
        total_ram = virtual_mem.total

        # Convert to human-readable format (e.g., GB)
        print(f"💾 Total RAM:     {humanize.naturalsize(total_ram)}")
        print(f"✅ Available RAM: {humanize.naturalsize(available_ram)}")
        print(f"📦 Used RAM:      {humanize.naturalsize(used_ram)}")

        print("Step 4: Historical CTR features")
        df = self.create_historical_ctr_features(df)
         # Get memory info
        virtual_mem = psutil.virtual_memory()

        # Available and used memory in bytes
        available_ram = virtual_mem.available
        used_ram = virtual_mem.used
        total_ram = virtual_mem.total

        # Convert to human-readable format (e.g., GB)
        print(f"💾 Total RAM:     {humanize.naturalsize(total_ram)}")
        print(f"✅ Available RAM: {humanize.naturalsize(available_ram)}")
        print(f"📦 Used RAM:      {humanize.naturalsize(used_ram)}")

        print("Step 5: Offer metadata features")
        df = self.create_offer_metadata_features(df)
         # Get memory info
        virtual_mem = psutil.virtual_memory()

        # Available and used memory in bytes
        available_ram = virtual_mem.available
        used_ram = virtual_mem.used
        total_ram = virtual_mem.total

        # Convert to human-readable format (e.g., GB)
        print(f"💾 Total RAM:     {humanize.naturalsize(total_ram)}")
        print(f"✅ Available RAM: {humanize.naturalsize(available_ram)}")
        print(f"📦 Used RAM:      {humanize.naturalsize(used_ram)}")

        print("Step 6: Transaction features")
        df = self.create_transaction_features(df)
         # Get memory info
        virtual_mem = psutil.virtual_memory()

        # Available and used memory in bytes
        available_ram = virtual_mem.available
        used_ram = virtual_mem.used
        total_ram = virtual_mem.total

        # Convert to human-readable format (e.g., GB)
        print(f"💾 Total RAM:     {humanize.naturalsize(total_ram)}")
        print(f"✅ Available RAM: {humanize.naturalsize(available_ram)}")
        print(f"📦 Used RAM:      {humanize.naturalsize(used_ram)}")

        print("Step 7: User-offer interaction features")
        df = self.create_user_offer_interaction_features(df)
         # Get memory info
        virtual_mem = psutil.virtual_memory()

        # Available and used memory in bytes
        available_ram = virtual_mem.available
        used_ram = virtual_mem.used
        total_ram = virtual_mem.total

        # Convert to human-readable format (e.g., GB)
        print(f"💾 Total RAM:     {humanize.naturalsize(total_ram)}")
        print(f"✅ Available RAM: {humanize.naturalsize(available_ram)}")
        print(f"📦 Used RAM:      {humanize.naturalsize(used_ram)}")

        print("Step 8: Customer engagement features")
        df = self.create_customer_engagement_features(df)
         # Get memory info
        virtual_mem = psutil.virtual_memory()

        # Available and used memory in bytes
        available_ram = virtual_mem.available
        used_ram = virtual_mem.used
        total_ram = virtual_mem.total

        # Convert to human-readable format (e.g., GB)
        print(f"💾 Total RAM:     {humanize.naturalsize(total_ram)}")
        print(f"✅ Available RAM: {humanize.naturalsize(available_ram)}")
        print(f"📦 Used RAM:      {humanize.naturalsize(used_ram)}")

        print("Step 9: Industry-specific features")
        df = self.create_industry_specific_features(df)
         # Get memory info
        virtual_mem = psutil.virtual_memory()

        # Available and used memory in bytes
        available_ram = virtual_mem.available
        used_ram = virtual_mem.used
        total_ram = virtual_mem.total

        # Convert to human-readable format (e.g., GB)
        print(f"💾 Total RAM:     {humanize.naturalsize(total_ram)}")
        print(f"✅ Available RAM: {humanize.naturalsize(available_ram)}")
        print(f"📦 Used RAM:      {humanize.naturalsize(used_ram)}")

        print("Step 10: Ranking features")
        df = self.create_ranking_features(df)
         # Get memory info
        virtual_mem = psutil.virtual_memory()

        # Available and used memory in bytes
        available_ram = virtual_mem.available
        used_ram = virtual_mem.used
        total_ram = virtual_mem.total

        # Convert to human-readable format (e.g., GB)
        print(f"💾 Total RAM:     {humanize.naturalsize(total_ram)}")
        print(f"✅ Available RAM: {humanize.naturalsize(available_ram)}")
        print(f"📦 Used RAM:      {humanize.naturalsize(used_ram)}")

        print("Step 11: Interaction features")
        df = self.create_interaction_features(df)
         # Get memory info
        virtual_mem = psutil.virtual_memory()

        # Available and used memory in bytes
        available_ram = virtual_mem.available
        used_ram = virtual_mem.used
        total_ram = virtual_mem.total

        # Convert to human-readable format (e.g., GB)
        print(f"💾 Total RAM:     {humanize.naturalsize(total_ram)}")
        print(f"✅ Available RAM: {humanize.naturalsize(available_ram)}")
        print(f"📦 Used RAM:      {humanize.naturalsize(used_ram)}")

        return df

    def prepare_features(self, df, is_training=True):
        """Complete + memory-efficient feature preparation pipeline"""
        print("🔧 Starting feature preparation...")

        # For large data: process in chunks
        if len(df) > 1_000_000:
            chunks = []
            for i in range(0, len(df), 500_000):
                print(f"🔧 Processing chunk {i//500_000 + 1}/{(len(df)//500_000)+1}")
                chunk = df.iloc[i:i+500_000].copy()
                chunk = self._process_feature_chunk(chunk, is_training)
                chunks.append(chunk)
                del chunk
                gc.collect()
            df = pd.concat(chunks, axis=0)
            del chunks
            gc.collect()
        else:
            df = self._process_feature_chunk(df, is_training)

        is_training=True


        # Feature selection (only for training)
        if is_training and 'y' in df.columns:
            print("🧠 Performing feature selection...")

            # Step 1: Apply variance threshold
            selected_features = self.apply_variance_threshold(df, is_training)

            # Step 2: Prepare matrix
            X = df[selected_features].fillna(0)
            y = df['y']

            # Step 3: RandomForest-based selection
            clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
            clf.fit(X, y)

            selector = SelectFromModel(clf, threshold="median", prefit=True)
            X_selected = selector.transform(X)

            # Step 4: Save selected feature names
            self.feature_cols = X.columns[selector.get_support()].tolist()
            print(f"✅ Selected {len(self.feature_cols)} features using RandomForest importance")

            # Save selector
            joblib.dump(self.feature_selector, f"{self.temp_dir}feature_selector.pkl")

        elif not is_training:
            if os.path.exists(f"{self.temp_dir}feature_selector.pkl"):
                self.feature_selector = joblib.load(f"{self.temp_dir}feature_selector.pkl")

                if self.feature_selector is not None:
                    selected_features = self.apply_variance_threshold(df, is_training)
                    X = df[selected_features].fillna(0)
                    selected_mask = self.feature_selector.get_support()
                    self.feature_cols = [col for col, selected in zip(selected_features, selected_mask) if selected]
                else:
                    print("⚠️ feature_selector.pkl exists but is empty or corrupted — falling back to all numeric features")
                    self.feature_cols = df.select_dtypes(include=[np.number]).columns.tolist()
            else:
                print("⚠️ No selector found — using all numeric features")
                self.feature_cols = df.select_dtypes(include=[np.number]).columns.tolist()

        gc.collect()
        return df

    def _split_train_val_by_time(self, df, date_col='id5_dt', cutoff_date=None):
        """
        Split train/val based on a time cutoff on date_col.
        Ensures both splits are non-empty.
        """
        print("🔄 Creating time-based train-validation split...")

        # Ensure date format
        if df[date_col].dtype == object:
            df[date_col] = pd.to_datetime(df[date_col], errors='coerce')

        # Use provided cutoff_date or calculate it
        if cutoff_date is None:
            cutoff = df[date_col].quantile(0.8)
        else:
            cutoff = pd.to_datetime(cutoff_date)

        # Split
        train_split = df[df[date_col] < cutoff].copy()
        val_split = df[df[date_col] >= cutoff].copy()

        # Validate non-empty
        if len(train_split) == 0 or len(val_split) == 0:
            raise ValueError(f"❌ Invalid split: train={len(train_split)} val={len(val_split)}. Try adjusting cutoff date.")

        print(f"✅ Train split: {len(train_split)} rows, Val split: {len(val_split)} rows")
        return train_split, val_split

    def create_time_split(self, train_df):
        """Create time-based train-validation split using id5"""
        print("🔄 Creating time-based train-validation split...")

        # Convert id5 back to datetime for splitting
        train_df['id5_dt'] = pd.to_datetime(train_df['id5'], format='%m-%d-%Y', errors='coerce')

        # Check if conversion was successful
        if train_df['id5_dt'].isna().all():
            print("⚠️ Warning: All dates failed to convert. Trying different format...")
            train_df['id5_dt'] = pd.to_datetime(train_df['id5'], errors='coerce')

        # Remove rows with invalid dates
        valid_dates = ~train_df['id5_dt'].isna()
        if valid_dates.sum() == 0:
            raise ValueError("❌ No valid dates found in id5 column")

        train_df = train_df[valid_dates].copy()

        # Use 80% earliest dates for training, 20% latest for validation
        split_date = train_df['id5_dt'].quantile(0.8)
        print(f"📅 Split date: {split_date}")

        # Call the split function with the calculated cutoff
        train_split, val_split = self._split_train_val_by_time(train_df, date_col='id5_dt', cutoff_date=split_date)

        return train_split, val_split

    def optimize_lightgbm_params(self, train_split, val_split):
        """Optimize LightGBM parameters for binary classification using Optuna"""
        print("🔄 Optimizing LightGBM parameters for classification...")

        def objective(trial):
            # Suggest hyperparameters for binary classification
            params = {
                'objective': 'binary',
                'boosting_type': 'gbdt',
                'metric': 'auc',
                'num_leaves': trial.suggest_int('num_leaves', 31, 200),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
                'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
                'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
                'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
                'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
                'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
                'verbose': -1,
                'seed': 42,
                'n_jobs': -1
            }
            # In optimize_lightgbm_params()
            params['feature_fraction'] = trial.suggest_float('feature_fraction', 0.4, 1.0)
            params['top_rate'] = trial.suggest_float('top_rate', 0.1, 0.5)  # GOSS sampling
            params['other_rate'] = trial.suggest_float('other_rate', 0.1, 0.5)

            try:
                X_train = train_split[self.feature_cols].fillna(0)
                y_train = train_split['y']
                X_val = val_split[self.feature_cols].fillna(0)
                y_val = val_split['y']

                train_dataset = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
                val_dataset = lgb.Dataset(X_val, label=y_val, reference=train_dataset, free_raw_data=False)

                model = lgb.train(
                    params,
                    train_dataset,
                    num_boost_round=1000,
                    valid_sets=[val_dataset],
                    valid_names=['valid'],
                    callbacks=[lgb.early_stopping(100, verbose=False)]
                )

                return model.best_score['valid']['auc']

            except Exception as e:
                print(f"Trial failed with error: {e}")
                return 0.0

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=10)

        best_params = study.best_params
        best_params.update({
            'objective': 'binary',
            'boosting_type': 'gbdt',
            'metric': 'auc',
            'verbose': -1
        })

        print(f"✅ Best LightGBM parameters: {best_params}")
        return best_params

    def train_lightgbm_classifier(self, train_df, sampling_method='none'):
        """Train LightGBM binary classification model with sampling"""
        print("🚀 Training LightGBM classification ensemble...")

        self.models['lightgbm_ensemble'] = []

        # Create time-based split for parameter tuning
        train_split, val_split = self.create_time_split(train_df)

        # Optimize parameters for classification
        best_params = self.optimize_lightgbm_params(train_split, val_split)

        # Prepare full training data
        X_train = train_split[self.feature_cols].fillna(0)
        y_train = train_split['y']

        # Apply sampling
        X_train, y_train = self.apply_sampling(X_train, y_train, sampling_method=sampling_method)

        # Train an ensemble of models with different seeds for stability
        for i in range(3):  # Reduced from 5 to 3 for efficiency
            print(f"🔄 Training LightGBM model {i+1}/3...")

            model_params = best_params.copy()
            model_params['seed'] = 42 + i

            try:
                train_dataset = lgb.Dataset(X_train, label=y_train, free_raw_data=False)

                model = lgb.train(
                    model_params,
                    train_dataset,
                    num_boost_round=1800,
                    valid_sets=[train_dataset],
                    callbacks=[
                        lgb.early_stopping(150, verbose=False),
                        lgb.log_evaluation(period=200)
                    ]
                )

                self.models['lightgbm_ensemble'].append(model)
                print(f"✅ Model {i+1} trained successfully")
            except Exception as e:
                print(f"❌ Failed to train model {i+1}: {e}")

        print(f"✅ LightGBM ensemble trained with {len(self.models['lightgbm_ensemble'])} models")

    def optimize_xgboost_params(self, train_split, val_split):
        """Optimize XGBoost parameters using Optuna"""
        print("🔄 Optimizing XGBoost parameters...")

        try:
            import xgboost as xgb

            def objective(trial):
                params = {
                    'objective': 'rank:pairwise',  # Changed from rank:map
                    'eval_metric': 'ndcg@7',      # Changed from map@7
                    'eta': trial.suggest_float('eta', 0.01, 0.1),
                    'gamma': trial.suggest_float('gamma', 0.0, 2.0),
                    'max_depth': trial.suggest_int('max_depth', 6, 12),
                    'min_child_weight': trial.suggest_int('min_child_weight', 20, 100),
                    'subsample': trial.suggest_float('subsample', 0.6, 1.0),
                    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
                    'tree_method': 'hist',
                    'seed': 42
                }

                # Prepare training data
                X_train = train_split[self.feature_cols].fillna(0)
                y_train = train_split['y']
                groups_train = train_split['id2'].astype(int)
                group_sizes_train = groups_train.value_counts().sort_index().values

                # Prepare validation data
                X_val = val_split[self.feature_cols].fillna(0)
                y_val = val_split['y']
                groups_val = val_split['id2'].astype(int)
                group_sizes_val = groups_val.value_counts().sort_index().values

                # Create DMatrix
                dtrain = xgb.DMatrix(X_train, label=y_train)
                dtrain.set_group(group_sizes_train)

                dval = xgb.DMatrix(X_val, label=y_val)
                dval.set_group(group_sizes_val)

                # Train model
                model = xgb.train(
                    params,
                    dtrain,
                    num_boost_round=200,
                    evals=[(dval, 'val')],
                    early_stopping_rounds=30,
                    verbose_eval=0
                )

                # Get validation score
                val_score = model.best_score
                return val_score

            # Run optimization
            study = optuna.create_study(direction='maximize')
            study.optimize(objective, n_trials=15)

            best_params = study.best_params
            best_params.update({
                'objective': 'rank:pairwise',
                'eval_metric': 'ndcg@7',
                'tree_method': 'hist'
            })

            print(f"✅ XGBoost Best parameters: {best_params}")
            print(f"✅ XGBoost Best score: {study.best_value}")

            return best_params

        except Exception as e:
            print(f"⚠️ XGBoost optimization failed: {str(e)}")
            # Return default params
            return {
                'objective': 'rank:pairwise',
                'eval_metric': 'ndcg@7',
                'eta': 0.05,
                'gamma': 1.0,
                'max_depth': 8,
                'min_child_weight': 50,
                'subsample': 0.8,
                'colsample_bytree': 0.8,
                'tree_method': 'hist',
                'seed': 42
            }

    def train_xgboost_classifier(self, train_df):
        """Train XGBoost binary classification model"""
        try:
            import xgboost as xgb
            print("🚀 Training XGBoost Classifier...")

            # For simplicity, we'll use fixed, robust parameters for XGBoost
            # You can build an optimization function similar to LightGBM's if needed
            params = {
                'objective': 'binary:logistic',
                'eval_metric': 'auc',
                'eta': 0.05,
                'max_depth': 7,
                'subsample': 0.8,
                'colsample_bytree': 0.8,
                'min_child_weight': 50,
                'tree_method': 'hist',
                'seed': 42
            }

            X_train = train_df[self.feature_cols].fillna(0)
            y_train = train_df['y']

            # Create DMatrix (no groups needed for classification)
            dtrain = xgb.DMatrix(X_train, label=y_train)

            model = xgb.train(
                params,
                dtrain,
                num_boost_round=1800,
                evals=[(dtrain, 'train')],
                early_stopping_rounds=150,
                verbose_eval=100
            )

            self.models['xgboost'] = model
            print("✅ XGBoost training completed")

        except ImportError:
            print("⚠️ XGBoost not installed. Skipping XGBoost training.")
            self.models['xgboost'] = None
        except Exception as e:
            print(f"⚠️ XGBoost training failed: {str(e)}")
            self.models['xgboost'] = None

    def generate_r3_submission(self, test_data_path="/kaggle/input/test-data/test_data_r3.parquet"):
        """
        Special method to generate submission for round 3
        Args:
            test_data_path: Path to the R3 test data parquet file
        """
        print("🚀 Generating submission for Round 3")

        try:
            # Load R3 test data
            print(f"🔄 Loading test data from {test_data_path}")
            test_df = pd.read_parquet(test_data_path)

            # Prepare features (use is_training=False)
            print("🔧 Preparing features...")
            test_df = self.prepare_features(test_df, is_training=False)

            # Generate submission
            submission = self.generate_submission(test_df, "submission_r3.csv")

            return submission

        except Exception as e:
            print(f"❌ R3 submission failed: {str(e)}")
            import traceback
            traceback.print_exc()
            return None

    def generate_submission(self, test_df, output_file="submission_r3.csv"):
        """
        Generate submission file with all required columns filled
        Args:
            test_df: DataFrame from test_data_r3.parquet
            output_file: Name of submission file to generate
        """
        print(f"📤 Generating submission file: {output_file}")

        try:
            # --- FIX: Ensure test features match training features ---
            # 1. Get the columns the model was trained on
            trained_features = self.feature_cols

            # 2. Get the columns currently in the test dataframe
            test_features = test_df.columns.tolist()

            # 3. Identify missing columns
            missing_cols = set(trained_features) - set(test_features)
            if missing_cols:
                print(f"⚠️ Found {len(missing_cols)} missing columns in test data: {list(missing_cols)}")
                # Add missing columns and fill with a neutral value like 0
                for c in missing_cols:
                    test_df[c] = 0.5

            # Get predictions from all models
            predictions = []

            # LightGBM ensemble predictions
            if 'lightgbm_ensemble' in self.models and self.models['lightgbm_ensemble']:
                print("🔄 Predicting with LightGBM ensemble...")
                X_test = test_df[self.feature_cols].fillna(0)
                lgb_preds = []
                for model in self.models['lightgbm_ensemble']:
                    try:
                        pred = model.predict(X_test)
                        lgb_preds.append(pred)
                    except Exception as e:
                        print(f"⚠️ LightGBM prediction error: {str(e)}")
                if lgb_preds:
                    predictions.append(np.mean(lgb_preds, axis=0))

            # XGBoost predictions if available
            if 'xgboost' in self.models and self.models['xgboost']:
                print("🔄 Predicting with XGBoost...")
                try:
                    import xgboost as xgb
                    dtest = xgb.DMatrix(test_df[self.feature_cols].fillna(0))
                    xgb_pred = self.models['xgboost'].predict(dtest)
                    predictions.append(xgb_pred)
                except Exception as e:
                    print(f"⚠️ XGBoost prediction error: {str(e)}")

            if not predictions:
                raise RuntimeError("No valid predictions were generated")

            # Ensemble predictions (simple average)
            final_pred = np.mean(predictions, axis=0)

            # Create submission dataframe with all required columns
            submission = pd.DataFrame({
                'id1': test_df['id1'],
                'id2': test_df['id2'],
                'id3': test_df['id3'],
                'id5': test_df['id5'],
                'pred': final_pred.astype(np.float32)
            })

            # Format datetime columns consistently

            if pd.api.types.is_datetime64_any_dtype(submission['id5']):
                submission['id5'] = submission['id5'].dt.strftime('%d-%m-%Y')

            # Save to CSV
            submission.to_csv(output_file, index=False)
            print(f"✅ Submission saved to {output_file} with {len(submission)} rows")
            print(f"📊 Prediction stats - Mean: {final_pred.mean():.4f}, Std: {final_pred.std():.4f}")

            return submission

        except Exception as e:
            print(f"❌ Failed to generate submission: {str(e)}")
            import traceback
            traceback.print_exc()
            return None

    def run_pipeline(self, generate_r3_submission=False, sampling_method='none'):
        """Complete pipeline execution with sampling option"""
        print(f"🚀 Starting AMEX Click Prediction Pipeline with {sampling_method} sampling...")

        try:
            # 1. Load data
            self.load_data()

            # 2. Prepare training features
            print("🔧 Preparing training features...")
            train_df = self.prepare_features(self.train_data, is_training=True)

            # 3. Train models with sampling
            print("🎯 Training models...")
            self.train_lightgbm_classifier(train_df, sampling_method=sampling_method)
            self.train_xgboost_classifier(train_df)

            # 4. Feature importance analysis
            self.display_feature_importance(train_df)

            # 5. Generate submission
            if generate_r3_submission:
                self.generate_r3_submission()
            else:
                print("🔧 Preparing test features...")
                test_df = self.prepare_features(self.test_data, is_training=False)
                self.generate_submission(test_df)

            print("✅ Pipeline completed successfully!")

        except Exception as e:
            print(f"❌ Pipeline failed: {str(e)}")
            import traceback
            traceback.print_exc()

# Execute pipeline
if __name__ == "__main__":
    pipeline = AMEXClickPredictionPipeline()
    pipeline.run_pipeline(generate_r3_submission=True, sampling_method='smote')